In [1]:
import osmnx as ox
from pprint import pprint
import networkx as nx
import igraph as ig
import os
from tqdm.auto import tqdm
from collections import deque
os.chdir('../')

Misc testing

In [2]:
from roc_bike_growth.loader import carall_from_polygon, bike_infra_from_polygon

In [3]:
rochester = ox.geocode_to_gdf('rochester, ny').geometry[0]
# carall = carall_from_polygon(rochester, add_pois=True)
bike_infra = bike_infra_from_polygon(rochester)
    

C:\Users\hoang\miniconda3\envs\ox\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


In [ ]:
edges = bike_infra.edges()
nodes = list(bike_infra.nodes())
adjacency_mat = nx.adjacency_matrix(bike_infra)

In [ ]:
print(adjacency_mat.shape)

In [ ]:
running_largest = []
G_new = bike_infra.to_undirected(as_view=False)
print(len(list(G_new.edges())))
edges = list(G_new.edges())

for edge in tqdm(edges):
    G_new.remove_edge(edge[0], edge[1])
    largest_component = max(nx.connected_components(G_new), key=len)
    running_largest.append(len(largest_component))
    
print(running_largest)
    
    

Pipeline

metrics self implemented / package

In [2]:

def graph_resilience(G, variant = 'density'):
    assert variant in ['density', 'largest_component']
    if variant == 'density':
        return round(nx.density(G)*100, 2)
    elif variant == 'largest_component':
        G_new = G.to_undirected(as_view=False)
        return len(max(nx.connected_components(G_new), key=len))

def graph_cohesion(G, coverage):
    assert isinstance(coverage, float) 
    G_new = G.to_undirected(as_view=False)
    n_components = nx.number_connected_components(G_new)
    return round(coverage / (n_components**1.2 + 0.00001), 2)

def graph_global_efficiency(G):
    G_new = G.to_undirected(as_view=False)
    return nx.algorithms.efficiency_measures.global_efficiency(G_new)

def graph_local_efficiency(G):
    G_new = G.to_undirected(as_view=False)
    return nx.algorithms.efficiency_measures.local_efficiency(G_new)


metric paper

In [16]:
import random
from haversine import haversine_vector
import itertools
import numpy as np
import json

In [4]:

def dist_vector(v1_list, v2_list):
    dist_list = haversine_vector(v1_list, v2_list, unit="m") # [(lat,lon)], [(lat,lon)]
    return dist_list

def paper_global_efficiency(G, pairs_thresh=500):
    # Input is a igraph Graph
    try:
        G = ig.Graph.from_networkx(nx.Graph(G))
    except:
        pass
    
    if G.vcount() > pairs_thresh:
        nodeindices = random.sample(list(G.vs.indices), pairs_thresh)
    else:
        nodeindices = list(G.vs.indices)
    
    d_ij = G.shortest_paths(source = nodeindices, target = nodeindices)
    d_ij = [item for sublist in d_ij for item in sublist] # flatten
    EG  = sum([1/d for d in d_ij if d != 0])
    pairs = list(itertools.permutations(nodeindices, 2))
    l_ij = dist_vector([(G.vs[p[0]]["y"], G.vs[p[0]]["x"]) for p in pairs],
                            [(G.vs[p[1]]["y"], G.vs[p[1]]["x"]) for p in pairs]) # must be in format lat,lon = y,x
    EG_id = sum([1/l for l in l_ij if l != 0])
    
    return round(EG / EG_id, 2)

In [11]:
def paper_local_efficiency(G, numnodepairs=500):
    # Input is a igraph Graph
    try:
        G = ig.Graph.from_networkx(nx.Graph(G))
    except:
        pass
    
    if G.vcount() > numnodepairs:
        nodeindices = random.sample(list(G.vs.indices), numnodepairs)
    else:
        nodeindices = list(G.vs.indices)
    EGi = []
    for i in nodeindices:
        if len(G.neighbors(i)) > 1: # If we have a nontrivial neighborhood
            G_induced = G.induced_subgraph(G.neighbors(i))
            EGi.append(paper_global_efficiency(G_induced, numnodepairs))
    EGi = sum(EGi) / len(EGi)
    
    return round(EGi, 2)

In [12]:
import pyproj
from shapely import ops
from shapely.geometry import Polygon, LineString
import copy

def paper_coverage(G):
    G = ig.Graph.from_networkx(nx.Graph(G))
    G_added = copy.deepcopy(G)
    
    # https://gis.stackexchange.com/questions/121256/creating-a-circle-with-radius-in-metres
    longitudes = [v["x"] for v in G.vs]
    loncenter = sum(longitudes) / (len(longitudes) + 0.0001)
    latitudes = [v["y"] for v in G.vs]
    latcenter = sum(latitudes) / (len(latitudes) + 0.0001)
    local_azimuthal_projection = "+proj=aeqd +R=6371000 +units=m +lat_0={} +lon_0={}".format(latcenter, loncenter)
    
    # Use transformer: https://gis.stackexchange.com/questions/127427/transforming-shapely-polygon-and-multipolygon-objects
    wgs84_to_aeqd = pyproj.Transformer.from_proj(
        pyproj.Proj("+proj=longlat +datum=WGS84 +no_defs"),
        pyproj.Proj(local_azimuthal_projection))
    aeqd_to_wgs84 = pyproj.Transformer.from_proj(
        pyproj.Proj(local_azimuthal_projection),
        pyproj.Proj("+proj=longlat +datum=WGS84 +no_defs"))
    edgetuples = [((e.source_vertex["x"], e.source_vertex["y"]), (e.target_vertex["x"], e.target_vertex["y"])) for e in G_added.es]
    
    # # Shapely buffer seems slow for complex objects: https://stackoverflow.com/questions/57753813/speed-up-shapely-buffer
    # # Therefore we buffer piecewise.
    cov_added = Polygon()
    for c, t in enumerate(edgetuples):
        buf = ops.transform(aeqd_to_wgs84.transform, ops.transform(wgs84_to_aeqd.transform, LineString(t)).buffer(500))
        cov_added = ops.unary_union([cov_added, Polygon(buf)])

    cov_transformed = ops.transform(wgs84_to_aeqd.transform, cov_added)
    covered_area = cov_transformed.area / 1000000 # turn from m2 to km2

    return covered_area

In [7]:
import osmnx as ox
import networkx as nx
import igraph as ig
import pandas as pd
import plotly.express as px

from roc_bike_growth.loader import POI_graph_from_polygon, bike_infra_from_polygon, carall_from_polygon
from roc_bike_growth.paper_gt import gt_with_existing_full

class BikeGraph:
    def __init__(self, prune_factor=0.1, route_factor=1):
        self.prune_factor = prune_factor
        self.route_factor = route_factor
        car_infra, bike_infra = self.get_data()
        self.bike_graph = self.merge_and_gt(car_infra, bike_infra)
        
    def get_data(self):
        rochester = ox.geocode_to_gdf('rochester, ny').geometry[0]
        bike_infra = bike_infra_from_polygon(rochester)
        car_infra = carall_from_polygon(rochester, add_pois=True)
        
        return car_infra, bike_infra
    
    def merge_and_gt(self, car_infra, bike_infra):
        return gt_with_existing_full(car_infra, bike_infra, self.route_factor, self.prune_factor)
    
    def plot_graph(self, add_pois=False):
        fig, ax = ox.plot.plot_graph(self.bike_graph)
        # Plot POIs on graph
        if add_pois:
            x, y = [], []
            pois = nx.get_node_attributes(self.bike_graph, 'poi').keys()
            for node in pois:
                d = self.bike_graph.nodes()[node]
                x.append(d['x'])
                y.append(d['y'])
            ax.scatter(x,y)
            fig
    
    def display_final_metrics(self):
        res_density = graph_resilience(self.bike_graph, 'density')
        res_component = graph_resilience(self.bike_graph, 'largest_component')
        coverage = paper_coverage(self.bike_graph)
        cohesion = graph_cohesion(self.bike_graph, coverage)
        global_eff = paper_global_efficiency(self.bike_graph)
        local_eff = paper_local_efficiency(self.bike_graph)
                
        plot_data = pd.DataFrame(dict(
           theta = ['resilience (density)','resilience (largest_component)','coverage','cohesion', 'global efficiency', 'local efficiency'],
           r = [res_density, res_component, coverage, cohesion, global_eff, local_eff]
        ))
        fig = px.line_polar(plot_data, r='r', theta='theta', line_close=True)
        fig.show()
        
        

In [9]:
roc_bike_graph = BikeGraph(prune_factor=1, route_factor=1)

C:\Users\hoang\miniconda3\envs\ox\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


Exception at Port of Rochester, Lake Ave . This point will be dropped:
 Nominatim could not geocode query "Port of Rochester, Lake Ave  rochester ny"
Exception at 810-846 N GOODMAN ST. This point will be dropped:
 Nominatim could not geocode query "810-846 N GOODMAN ST rochester ny"
Exception at 650-672 E Main St. This point will be dropped:
 Nominatim could not geocode query "650-672 E Main St rochester ny"
Exception at 497-499 S. CLINTON. This point will be dropped:
 Nominatim could not geocode query "497-499 S. CLINTON rochester ny"


d:\work\grad\y1s2\capstone\project\roc-bike-growth\roc_bike_growth\paper_gt.py:23: RuntimeWarning: Couldn't reach some vertices at src/paths/unweighted.c:368
  poi_nodes.append(G.get_shortest_paths(v, pois_indices[c:], output="vpath"))
d:\work\grad\y1s2\capstone\project\roc-bike-growth\roc_bike_growth\paper_gt.py:24: RuntimeWarning: Couldn't reach some vertices at src/paths/unweighted.c:368
  poi_edges.append(G.get_shortest_paths(v, pois_indices[c:], output="epath"))


In [ ]:
roc_bike_graph.display_final_metrics()

In [ ]:
roc_bike_graph.plot_graph()

In [13]:
res_density = graph_resilience(roc_bike_graph.bike_graph, 'density')
res_component = graph_resilience(roc_bike_graph.bike_graph, 'largest_component')
coverage = paper_coverage(roc_bike_graph.bike_graph)
cohesion = graph_cohesion(roc_bike_graph.bike_graph, coverage)
global_eff = paper_global_efficiency(roc_bike_graph.bike_graph)
local_eff = paper_local_efficiency(roc_bike_graph.bike_graph)

print(res_density, res_component, coverage, cohesion, global_eff, local_eff)

# plot_data = pd.DataFrame(dict(
#     theta = ['resilience (density)','resilience (largest_component)','coverage','cohesion', 'global efficiency', 'local efficiency'],
#     r = [res_density, res_component / 286, coverage/100, cohesion, global_eff/100, local_eff/100]
# ))
# fig = px.line_polar(plot_data, r='r', theta='theta', line_close=True)
# fig.show()

0.11 1795 108.02641503884439 1.69 87.86 2.59


In [18]:
data = []

prune_fs = np.arange(0.1, 1, 0.1)
route_fs = np.arange(0.1, 1, 0.1)
for prune_f in tqdm(prune_fs):
    for route_f in route_fs:
        bike_map = BikeGraph(prune_factor=prune_f, route_factor=route_f)
        
        res_density = graph_resilience(bike_map.bike_graph, 'density')
        res_component = graph_resilience(bike_map.bike_graph, 'largest_component')
        coverage = paper_coverage(bike_map.bike_graph)
        cohesion = graph_cohesion(bike_map.bike_graph, coverage)
        global_eff = paper_global_efficiency(bike_map.bike_graph)
        local_eff = paper_local_efficiency(bike_map.bike_graph)
        
        data.append({
            'prune_f': prune_f,
            'route_f': route_f,
            'res_density': res_density,
            'res_component': res_component,
            'coverage': coverage,
            'cohesion': cohesion,
            'global_eff': global_eff/100,
            'local_eff': local_eff/100
        })
        
with open('output\\results.json', 'w') as f:
    json.dump(data, f)


  0%|          | 0/9 [00:00<?, ?it/s]C:\Users\hoang\miniconda3\envs\ox\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


Exception at Port of Rochester, Lake Ave . This point will be dropped:
 Nominatim could not geocode query "Port of Rochester, Lake Ave  rochester ny"
Exception at 810-846 N GOODMAN ST. This point will be dropped:
 Nominatim could not geocode query "810-846 N GOODMAN ST rochester ny"
Exception at 650-672 E Main St. This point will be dropped:
 Nominatim could not geocode query "650-672 E Main St rochester ny"
Exception at 497-499 S. CLINTON. This point will be dropped:
 Nominatim could not geocode query "497-499 S. CLINTON rochester ny"


d:\work\grad\y1s2\capstone\project\roc-bike-growth\roc_bike_growth\paper_gt.py:23: RuntimeWarning: Couldn't reach some vertices at src/paths/unweighted.c:368
  poi_nodes.append(G.get_shortest_paths(v, pois_indices[c:], output="vpath"))
d:\work\grad\y1s2\capstone\project\roc-bike-growth\roc_bike_growth\paper_gt.py:24: RuntimeWarning: Couldn't reach some vertices at src/paths/unweighted.c:368
  poi_edges.append(G.get_shortest_paths(v, pois_indices[c:], output="epath"))
C:\Users\hoang\miniconda3\envs\ox\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


Exception at Port of Rochester, Lake Ave . This point will be dropped:
 Nominatim could not geocode query "Port of Rochester, Lake Ave  rochester ny"
Exception at 810-846 N GOODMAN ST. This point will be dropped:
 Nominatim could not geocode query "810-846 N GOODMAN ST rochester ny"
Exception at 650-672 E Main St. This point will be dropped:
 Nominatim could not geocode query "650-672 E Main St rochester ny"
Exception at 497-499 S. CLINTON. This point will be dropped:
 Nominatim could not geocode query "497-499 S. CLINTON rochester ny"


d:\work\grad\y1s2\capstone\project\roc-bike-growth\roc_bike_growth\paper_gt.py:23: RuntimeWarning: Couldn't reach some vertices at src/paths/unweighted.c:368
  poi_nodes.append(G.get_shortest_paths(v, pois_indices[c:], output="vpath"))
d:\work\grad\y1s2\capstone\project\roc-bike-growth\roc_bike_growth\paper_gt.py:24: RuntimeWarning: Couldn't reach some vertices at src/paths/unweighted.c:368
  poi_edges.append(G.get_shortest_paths(v, pois_indices[c:], output="epath"))
C:\Users\hoang\miniconda3\envs\ox\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


...

Exception at Port of Rochester, Lake Ave . This point will be dropped:
 Nominatim could not geocode query "Port of Rochester, Lake Ave  rochester ny"
Exception at 810-846 N GOODMAN ST. This point will be dropped:
 Nominatim could not geocode query "810-846 N GOODMAN ST rochester ny"
Exception at 650-672 E Main St. This point will be dropped:
 Nominatim could not geocode query "650-672 E Main St rochester ny"
Exception at 497-499 S. CLINTON. This point will be dropped:
 Nominatim could not geocode query "497-499 S. CLINTON rochester ny"


d:\work\grad\y1s2\capstone\project\roc-bike-growth\roc_bike_growth\paper_gt.py:23: RuntimeWarning: Couldn't reach some vertices at src/paths/unweighted.c:368
  poi_nodes.append(G.get_shortest_paths(v, pois_indices[c:], output="vpath"))
d:\work\grad\y1s2\capstone\project\roc-bike-growth\roc_bike_growth\paper_gt.py:24: RuntimeWarning: Couldn't reach some vertices at src/paths/unweighted.c:368
  poi_edges.append(G.get_shortest_paths(v, pois_indices[c:], output="epath"))
C:\Users\hoang\miniconda3\envs\ox\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


Exception at Port of Rochester, Lake Ave . This point will be dropped:
 Nominatim could not geocode query "Port of Rochester, Lake Ave  rochester ny"
Exception at 810-846 N GOODMAN ST. This point will be dropped:
 Nominatim could not geocode query "810-846 N GOODMAN ST rochester ny"
Exception at 650-672 E Main St. This point will be dropped:
 Nominatim could not geocode query "650-672 E Main St rochester ny"
Exception at 497-499 S. CLINTON. This point will be dropped:
 Nominatim could not geocode query "497-499 S. CLINTON rochester ny"


d:\work\grad\y1s2\capstone\project\roc-bike-growth\roc_bike_growth\paper_gt.py:23: RuntimeWarning: Couldn't reach some vertices at src/paths/unweighted.c:368
  poi_nodes.append(G.get_shortest_paths(v, pois_indices[c:], output="vpath"))
d:\work\grad\y1s2\capstone\project\roc-bike-growth\roc_bike_growth\paper_gt.py:24: RuntimeWarning: Couldn't reach some vertices at src/paths/unweighted.c:368
  poi_edges.append(G.get_shortest_paths(v, pois_indices[c:], output="epath"))
C:\Users\hoang\miniconda3\envs\ox\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))
 11%|█         | 1/9 [03:48<30:30, 228.75s/it]


KeyboardInterrupt: 